In [91]:
# -*- coding: utf-8 -*-
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [92]:
tree_train = ET.parse('../data/raw/news_sentiment_romip2012/train/news_eval_train.xml')
root_train = tree_train.getroot()
tree_test = ET.parse('../data/raw/news_sentiment_romip2012/test/news_eval_test.xml')
root_test = tree_test.getroot()

In [93]:
train_data = []
train_target = []
test_data = []
test_target = []
evs = {'0':3, '+':2, '-':1}
for child in root_train:
    speech = child.find('speech').text
    ev = child.find('evaluation').text.strip()
    if ev in evs.keys():
        train_data.append(speech)
        train_target.append(evs[ev])
for child in root_test:
    speech = child.find('speech').text
    ev = child.find('evaluation').text.strip()
    if ev in evs.keys():
        test_data.append(speech)
        test_target.append(evs[ev])

In [94]:
import re
import pymorphy2 as pm2
morph = pm2.MorphAnalyzer()

In [95]:
def normalize_text(text):
    normal_forms = []
    for word in re.findall(ur'[а-я]+', text.lower()):
        parsed = morph.parse(word)[0]
        norm_form = parsed.normal_form
        normal_forms.append(norm_form)
    normal_text = ' '.join(normal_forms)
    return normal_text

In [96]:
from joblib import Parallel, delayed  
import multiprocessing
num_cores = multiprocessing.cpu_count()

In [97]:
train_data = Parallel(n_jobs=num_cores)(delayed(normalize_text)(text) for text in train_data)

In [98]:
test_data = Parallel(n_jobs=num_cores)(delayed(normalize_text)(text) for text in test_data)

In [99]:
import codecs

In [100]:
train_f = codecs.open('../data/interim/clear_train.csv', 'w', 'utf-8')
for idx, val in tqdm(enumerate(train_data)):
    train_f.write(train_data[idx] + ',' + str(train_target[idx]) + '\n')
train_f.close()
test_f = codecs.open('../data/interim/clear_test.csv', 'w', 'utf-8')
for idx, val in tqdm(enumerate(test_data)):
    test_f.write(test_data[idx] + ',' + str(test_target[idx]) + '\n')
test_f.close()

3893it [00:00, 78784.99it/s]
4573it [00:00, 69456.25it/s]
